In [164]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pickle
from sklearn.cross_validation import train_test_split
import itertools
import nltk
from nltk.corpus import stopwords
import unicodedata
import sys
%matplotlib inline 

In [44]:
mypath = '/Users/Lucy/Google Drive/MSDS/2016Fall/DSGA1006/Data'
f = open(mypath + '/lstm-rnn/lstm_data.pickle','rb')
X_train, X_test, y_train, y_test = pickle.load(f)
f.close()

In [181]:
train_description = X_train.clean_description.copy()
test_description = X_test.clean_description.copy()
#train_description = [word for word in train_description if word not in stopwords.words('english')]
#test_description = [word for word in test_description if word not in stopwords.words('english')]

In [182]:
vocabulary_size = 50000
unknown_token = "UNKNOWNTOKEN"
sentence_start_token = "SENTENCESTART"
sentence_end_token = "SENTENCEEND"

In [183]:
# Split full descriptions into sentences
sentences = itertools.chain(*[[nltk.sent_tokenize(x.decode('utf-8').lower())] for x in train_description])
mod_sentences = []
for desc in sentences:
    full_desc = ""
    for x in desc:
        s = "%s %s %s " % (sentence_start_token, x, sentence_end_token)
        full_desc = full_desc + s
    mod_sentences.append(full_desc)

In [184]:
def strip_punctuation(text):
    """
    >>> strip_punctuation(u'something')
    u'something'

    >>> strip_punctuation(u'something.,:else really')
    u'somethingelse really'
    """
    punctutation_cats = set(['Pc', 'Pd', 'Ps', 'Pe', 'Pi', 'Pf', 'Po'])
    return ''.join(x for x in text
                   if unicodedata.category(x) not in punctutation_cats)

In [185]:
mod_sentences = map(strip_punctuation, mod_sentences)

In [186]:
tokenized_sentences = [nltk.word_tokenize(sent) for sent in mod_sentences]

In [187]:
# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print "Found %d unique words tokens." % len(word_freq.items())

Found 141209 unique words tokens.


In [193]:
word_freq = sorted(word_freq.items(), key=lambda i: i[1], reverse=True)

In [189]:
# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

print "Using vocabulary size %d." % vocabulary_size
print "The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1])

# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]
    


Using vocabulary size 50000.
The least frequent word in our vocabulary is 'smartmenucard' and appeared 2 times.


In [202]:
word_to_index.

[0, 1]

In [205]:
{word_to_index.keys()[word_to_index.values().index(k)]: k for k in sorted(word_to_index.values())[:2]}

{u'SENTENCEEND': 1, u'and': 0}

In [145]:
tokenized_sentences[0]

[u'SENTENCE_START',
 'UNKNOWN_TOKEN',
 u'technologies',
 u'is',
 u'a',
 u'sensor',
 u'and',
 u'services',
 u'company',
 u'focused',
 u'on',
 u'home',
 u'safety',
 u',',
 u'security',
 u',',
 u'and',
 u'loss',
 u'prevention',
 u'.',
 u'SENTENCE_END',
 u'SENTENCE_START',
 u'their',
 u'customers',
 u'use',
 u'their',
 u'devices',
 u',',
 u'in',
 u'connection',
 u'with',
 u'their',
 u'monitoring',
 u'service',
 u',',
 u'to',
 u'discover',
 u'and',
 u'alert',
 u'themselves',
 u'when',
 u'a',
 u'hazardous',
 u'event',
 u'occurs',
 u'in',
 u'their',
 u'home',
 u'.',
 u'SENTENCE_END',
 u'SENTENCE_START',
 'UNKNOWN_TOKEN',
 u'(',
 u'sensor',
 u'network',
 u'utilizing',
 u'powerline',
 u'infrastructure',
 u')',
 u'technology',
 u'is',
 u'an',
 u'ultra-low-power',
 u',',
 u'general-purpose',
 u'wireless',
 u'sensing',
 u'platform',
 u'that',
 u'is',
 u'easily',
 u'used',
 u'and',
 u'maintained',
 u'by',
 u'leveraging',
 u'the',
 u'existing',
 u'electrical',
 u'wiring',
 u'as',
 u'a',
 u'whole',
 

In [ ]:
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
